# Introduction

## Goals

By the end of this course, you should be able to
- Do basic data analysis using R or Python/Pandas, with a special emphasis on
  - triton, or other similar HPC cluster environments
  - workflows, I/O strategies etc. that work on HPC clusters.

What this course is NOT:
- A basic course in programming. We don't expect you to have prior knowledge of R or Python, but some programming experience is required.
- A basic course in statistics / machine learning. As part of the course we'll do some simple stuff, but we expect that you either understand the statistics from before or learn it on your own.

Topics that we're going to cover
- The dataframe data structure, and how it relates to other common data structures.
- Working with dataframes. Indexing, etc.
- Visualizing your results.

## Note about syntax

Anything written like `this` refers to R-code. Depending on context it might be a type or a function.

# Basic R structures and objects

## Libraries

R has a wide variety of libraries supplied in the R-CRAN-network (Comprehensive R Archive Network). Installing these libraries is usually as simple as running:

`install.package("packagename")`

Loading these libraries is done with:

`library(packagename)`

## Data types and variables

In R for the most of your time your data will be of the following types:
- `logical`is a boolean value with values `TRUE` OR `FALSE`.
- `integer` is a integer number. It is differentiated from generic numbers with an `L`-letter in the end. E.g. `3L`.
- `numeric` or `double` is a generic floating point number.
- `complex` refers to complex point numbers. One can specify them in the form `4 + 17i`.
- `character` refers both to individual characters and character strings.

Data type of an object can be found with `typeof`-function.

Variables in R are set with the assignment operator `<-`. `print`-function prints variables.

In [79]:
boolean_var <- FALSE
integer_var <- 73L
numeric_var <- 3.2
character_var <- 'Aalto'
complex_var <- 3+2i

typeof(boolean_var)
typeof(integer_var)
typeof(numeric_var)
typeof(character_var)
typeof(complex_var)

print(integer_var)

[1] "logical"

[1] "integer"

[1] "double"

[1] "character"

[1] "complex"

[1] 73


## Atomic vectors

R is a vectorized language. This means that most objects are `vector`-objects or built from `vector`s.

A atomic `vector` in R is a contiguous series of values where all values share a common data type. `vector`'s length is specified at its creation time.

To create a `vector` one can use the `vector`-function. It takes as its arguments the type and the length.

More often one uses the `c`-function (concatenate/combine) to combine data into a `vector`. When using this it is a good idea to make sure that types of the input data match or you will get a type conversion.

`str` (structure) can be used to get lots of readable information on vectors.

In [80]:
# Initialization using the vector-function
integer_vector <- vector('integer',5)
str(integer_vector)

# A vector of integers (note the L-letters)
integers <- c(1L, 2L, 5L, 7L)
str(integers)

# A vector of floating point numbers
floats <- c(7.5, 28.1, 0.3, 3.4)
str(floats)

# A vector containting character strings
characters <- c('fox', 'lynx', 'tiger', 'cat')
str(characters)

# A vector of generic numbers (note that type is not integer)
not_integers <- c(1, 2, 5, 7)
str(not_integers)

# Output determined by highest type in hierarchy:
# logical < integer < double < complex < character
type_conversion_warning <- c(TRUE,1L,7.5,'fox',1)
str(type_conversion_warning)

 int [1:5] 0 0 0 0 0
 int [1:4] 1 2 5 7
 num [1:4] 7.5 28.1 0.3 3.4
 chr [1:4] "fox" "lynx" "tiger" "cat"
 num [1:4] 1 2 5 7
 chr [1:5] "TRUE" "1" "7.5" "fox" "1"


### Vector indexing

Length of a `vector` can be obtained with function `length`.

`vector`s can be accessed by index using the `[start:finish]`-syntax, but typically one uses functions with `apply`-structures to access and modify them. More on that later.

One can also provide an indexing `vector` with the indices given as numbers. Indices in R start from 1.

In [81]:
# Value at index 2
integers[2]
# Values from index 2 to the end
floats[2:length(floats)]
# Values from index 2 to one before the end
characters[2:(length(characters)-1)]
# Values at indices 2 and 4 (note indexing with a vector)
not_integers[c(2,4)]

[1] 2

[1] 28.1  0.3  3.4

[1] "lynx"  "tiger"

[1] 2 7

Indexing with indices is quite rarely the way you want to access data as that forces you to keep track of data placement. More often you're interested in a certain range of data. For this you need logical indexing, where you create an indexing `vector` with a logical expression.

This indexing `vector` can then be used to slice the `vector`. Most of the time you don't need to store the indexing `vector`.

In [82]:
# Create an indexing vector
indexing_vector <- integers > 3
# Result is a boolean vector with the same dimensions as the original vector 
print(indexing_vector)
# This indexing vector can be used to index the original vector
integers[indexing_vector]

# Same as a one-liner
integers[integers > 3]

[1] FALSE FALSE  TRUE  TRUE


[1] 5 7

[1] 5 7

### Performance alert: Changing vector's size slows code down

One can edit the `vector` and their values after they have been created, but one needs to be mindful of the operations.

Some operations (like changes in values) are done *in place*. This means that the memory values are changed, but there was no need to create a new `vector`. This type of operation is fast and does not require additional memory.

On the other hand some operations (like appending) change the `vector` length. This forces the creation of a new `vector`, into which values from the old object are copied. This type of operation slows the code down and requires additional memory.

In R it is usually best to reserve a `vector` of the size you will use first and populate it with values afterwards.

## Factors

`factor`s are a special case of `vector`s that are used to store character strings in a more efficient way. This is especially important with data that can be split into categories.

In a `factor` each unique string in a `vector` is stored as an integer and the mapping between these integers and the original strings is stored as levels.

By using the `ordered=TRUE`-argument one can tell that data is ordered. With ordered `factor`s one can use logical indexing.

In [83]:
# R contains a constant list of month names
month.name

# Get a random sample of 10000 months from the month.name vector (e.g. birth months of participants in a study)
months <- sample(month.name, 10000, replace=T)

# Show months distribution in a table
table(months)

# Check how big the vector is in the memory
object.size(months)

# Now make the vector into a factor
months_f <- factor(months)

# Print month distribution in the table
table(months_f)

# Check how big the vector is in the memory
object.size(months_f)

# Make a new factor, but this time give the possible levels that data can take and tell that the levels are ordered
months_f2 <- factor(months, levels=month.name, ordered=TRUE)

# Print table (note that it is in order)
table(months_f2)

# Check how many of the months in the factor are on the second half of the year
sum(months_f2 > 'June')

[1] "January"   "February"  "March"     "April"     "May"       "June"     
 [7] "July"      "August"    "September" "October"   "November"  "December"

months
    April    August  December  February   January      July      June     March 
      799       809       803       806       849       844       875       872 
      May  November   October September 
      827       796       850       870 

80648 bytes

months_f
    April    August  December  February   January      July      June     March 
      799       809       803       806       849       844       875       872 
      May  November   October September 
      827       796       850       870 

41136 bytes

months_f2
  January  February     March     April       May      June      July    August 
      849       806       872       799       827       875       844       809 
September   October  November  December 
      870       850       796       803 

[1] 4972

## Lists

Sometimes you need a `vector` with mixed data types. In this case you need a `list`.

In [84]:
example_list <- list('a',3L,4.3,3+2i)
str(example_list)

List of 4
 $ : chr "a"
 $ : int 3
 $ : num 4.3
 $ : cplx 3+2i


`list` can also have named elements. During list creation they can be given as keyword arguments to the function call.

New elements can also be created with `$name`.

After `list` has been created there's quite a few ways to access its contents:
- `[ ]` returns a slice of the list that keeps the names intact. Thus it always returns a list.
- `[[ ]]` or `$` returns the value stored in key. Type of the object depends on the stored object.



In [85]:
# Initialize a list with various data types
example_list2 <- list(3L,42.4,something='nothing',c(13.2,32.2),boolean=TRUE, l=list(32.2,1.6))

# Add a new variable to the list
example_list2$variable <- 3.4

# Return a slice of the list
print('Slice:')
example_list2['something']
class(example_list2['something'])

# Return the value of 'something'
print('Value:')
example_list2[['something']]
class(example_list2[['something']])
example_list2$something
class(example_list2$something)

print('The whole list')

# Print the list. Note that the named values have indices.
print(example_list2)

[1] "Slice:"


$something
[1] "nothing"

[1] "list"

[1] "Value:"


[1] "nothing"

[1] "character"

[1] "nothing"

[1] "character"

[1] "The whole list"
[[1]]
[1] 3

[[2]]
[1] 42.4

$something
[1] "nothing"

[[4]]
[1] 13.2 32.2

$boolean
[1] TRUE

$l
$l[[1]]
[1] 32.2

$l[[2]]
[1] 1.6


$variable
[1] 3.4



### Performance alert: list is still a vector

The comment about `vector` size changes still matters with `list`s contents. If sizes of `vectors` contained in a `list` are modified, they need to be copied. Likewise if additional entries are added to the list, the whole list needs to be copied as well. 

## Matrices

`matrix` is two-dimensional `vector`. It can be created with the `matrix`-function. Matrices have dimensions instead of lenght and these can be accessed through `dim`-function.

In [86]:
# Using seq-function to create a sequence
example_matrix <- matrix(seq(0,5),nrow=3,ncol=2)
print(example_matrix)

dim(example_matrix)

     [,1] [,2]
[1,]    0    3
[2,]    1    4
[3,]    2    5


[1] 3 2

Two-dimensional indicing is done with `[ , ]`. Rows are to the left of comma, columns to the right. 

In [87]:
# Get first row (note that leaving columns empty means all columns)
example_matrix[1,]
# Get second column
example_matrix[,2]
# Get submatrix
example_matrix[2:3,1:2]

# One can also traverse the matrix in a flattened order (R uses column-major order)
example_matrix[4]

[1] 0 3

[1] 3 4 5

1,4
2,5


[1] 3

## Data frames

The `list`-object is much more usable than `vector`, but even more usable is the `data.frame`.

`data.frame` contains data in square tabular form, where each column is a observable and each row is an observation.[1](https://stat.ethz.ch/R-manual/R-devel/library/base/html/data.frame.html)

Let us load a internal R sample data set that has car information.

In [88]:
cars <- mtcars
is.data.frame(cars)

# Print only first rows
head(cars)

[1] TRUE

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2
Valiant,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1


The names of the columns and rows can be accessed through `colnames` and `rownames`. 

Number of columns and rows can be obtained with `ncol` and `nrow`.

In [89]:
colnames(cars)
rownames(cars)
ncol(cars)
nrow(cars)

[1] "mpg"  "cyl"  "disp" "hp"   "drat" "wt"   "qsec" "vs"   "am"   "gear"
[11] "carb"

[1] "Mazda RX4"           "Mazda RX4 Wag"       "Datsun 710"         
 [4] "Hornet 4 Drive"      "Hornet Sportabout"   "Valiant"            
 [7] "Duster 360"          "Merc 240D"           "Merc 230"           
[10] "Merc 280"            "Merc 280C"           "Merc 450SE"         
[13] "Merc 450SL"          "Merc 450SLC"         "Cadillac Fleetwood" 
[16] "Lincoln Continental" "Chrysler Imperial"   "Fiat 128"           
[19] "Honda Civic"         "Toyota Corolla"      "Toyota Corona"      
[22] "Dodge Challenger"    "AMC Javelin"         "Camaro Z28"         
[25] "Pontiac Firebird"    "Fiat X1-9"           "Porsche 914-2"      
[28] "Lotus Europa"        "Ford Pantera L"      "Ferrari Dino"       
[31] "Maserati Bora"       "Volvo 142E"

[1] 11

[1] 32

`data.frame` access is typically done through these names.

In [126]:
# Get specific row, all columns
cars["Dodge Challenger",]
# Get mpg for Dodge Challenger
cars[["Dodge Challenger","mpg"]]
# Get all rows, specific columns
cars[,c("cyl","disp")]
# Create a boolean vector where each row is TRUE or FALSE based on whether cyl == 6 or not
cyl6 <- cars[,"cyl"] == 6
cyl6
# Get all rows based on an indexing vector
cars[cyl6,]

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,l100km
Dodge Challenger,15.5,8,318,150,2.76,3.52,16.87,0,0,3,2,15.17513


[1] 15.5

,cyl,disp
Mazda RX4,6,160.0
Mazda RX4 Wag,6,160.0
Datsun 710,4,108.0
Hornet 4 Drive,6,258.0
Hornet Sportabout,8,360.0
Valiant,6,225.0
Duster 360,8,360.0
Merc 240D,4,146.7
Merc 230,4,140.8
Merc 280,6,167.6


[1]  TRUE  TRUE FALSE  TRUE FALSE  TRUE FALSE FALSE FALSE  TRUE  TRUE FALSE
[13] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[25] FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,l100km
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4,11.20069
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4,11.20069
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1,10.99134
Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1,12.99528
Merc 280,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4,12.25076
Merc 280C,17.8,6,167.6,123,3.92,3.440,18.90,1,0,4,4,13.21430
Ferrari Dino,19.7,6,145.0,175,3.62,2.770,15.50,0,1,5,6,11.93983


Let's create a new column from *mpg* that shows litre per 100 kilometres *lkm*: 

In [91]:
cars["l100km"] <- 235.214583 / cars["mpg"]
head(cars)

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb,l100km
Mazda RX4,21.0,6,160,110,3.90,2.620,16.46,0,1,4,4,11.20069
Mazda RX4 Wag,21.0,6,160,110,3.90,2.875,17.02,0,1,4,4,11.20069
Datsun 710,22.8,4,108,93,3.85,2.320,18.61,1,1,4,1,10.31643
Hornet 4 Drive,21.4,6,258,110,3.08,3.215,19.44,1,0,3,1,10.99134
Hornet Sportabout,18.7,8,360,175,3.15,3.440,17.02,0,0,3,2,12.57832
Valiant,18.1,6,225,105,2.76,3.460,20.22,1,0,3,1,12.99528


## Tibbles

`tibble` is a popular extension of `data.frame` and it is part of a famous [Tidyverse](https://www.tidyverse.org/)-flavor of R. It is meant to be simpler to use than `data.frame`. [[1]](http://tibble.tidyverse.org/)

In [92]:
library(tidyverse)

cars_tibble <- as_tibble(mtcars)

print(cars_tibble)

# A tibble: 32 x 11
     mpg   cyl  disp    hp  drat    wt  qsec    vs    am  gear  carb
 * <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1  21.0    6.  160.  110.  3.90  2.62  16.5    0.    1.    4.    4.
 2  21.0    6.  160.  110.  3.90  2.88  17.0    0.    1.    4.    4.
 3  22.8    4.  108.   93.  3.85  2.32  18.6    1.    1.    4.    1.
 4  21.4    6.  258.  110.  3.08  3.22  19.4    1.    0.    3.    1.
 5  18.7    8.  360.  175.  3.15  3.44  17.0    0.    0.    3.    2.
 6  18.1    6.  225.  105.  2.76  3.46  20.2    1.    0.    3.    1.
 7  14.3    8.  360.  245.  3.21  3.57  15.8    0.    0.    3.    4.
 8  24.4    4.  147.   62.  3.69  3.19  20.0    1.    0.    4.    2.
 9  22.8    4.  141.   95.  3.92  3.15  22.9    1.    0.    4.    2.
10  19.2    6.  168.  123.  3.92  3.44  18.3    1.    0.    4.    4.
# ... with 22 more rows


It is obvious that the `print` on a `tibble`is more informative than the one used a `data.frame`. Firsly, one does not need to use `head` to see just the beginning of data. Secondly, column data types and the overall size of the `tibble` are clearly visible.

`tibble` does some things differently to `data.frame`. Firsly, it never converts data types of variables (e.g. `vector`s of type `character` into `factor`s). Secondly, it does not generate row names. These changes are made to make the code more consistent and predictable. To add row names as a new column one needs to use `rownames_to_column`:

In [93]:
cars_tibble <- as_tibble(rownames_to_column(mtcars))
print(cars_tibble)

# A tibble: 32 x 12
   rowname      mpg   cyl  disp    hp  drat    wt  qsec    vs    am  gear  carb
   <chr>      <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1 Mazda RX4   21.0    6.  160.  110.  3.90  2.62  16.5    0.    1.    4.    4.
 2 Mazda RX4…  21.0    6.  160.  110.  3.90  2.88  17.0    0.    1.    4.    4.
 3 Datsun 710  22.8    4.  108.   93.  3.85  2.32  18.6    1.    1.    4.    1.
 4 Hornet 4 …  21.4    6.  258.  110.  3.08  3.22  19.4    1.    0.    3.    1.
 5 Hornet Sp…  18.7    8.  360.  175.  3.15  3.44  17.0    0.    0.    3.    2.
 6 Valiant     18.1    6.  225.  105.  2.76  3.46  20.2    1.    0.    3.    1.
 7 Duster 360  14.3    8.  360.  245.  3.21  3.57  15.8    0.    0.    3.    4.
 8 Merc 240D   24.4    4.  147.   62.  3.69  3.19  20.0    1.    0.    4.    2.
 9 Merc 230    22.8    4.  141.   95.  3.92  3.15  22.9    1.    0.    4.    2.
10 Merc 280    19.2    6.  168.  123.  3.92  3.44  18.3    1.    0.    4.    4.
# ... with 22 more r

Parts of a `tibble` are accessed using "verbs" of Tidyverse in conjuction with a pipe-structure.

Pipe `%>%` is used to minimize this kind of code:

```r
a <- data.frame(...)
a_tmp1 <- func1(a)
a_tmp2 <- func2(a_tmp1)
b <- func3(a_tmp2)
```

with `tibble` and `%>%` the previous structure would be

```r
a <- tibble(...)
b <- a %>% 
    func1() %>% 
    func2() %>%
    func3()
```

For data-access the imporant verbs are:
- `mutate()`, which adds new variables based on existing variables [[1]](http://dplyr.tidyverse.org/reference/mutate.html)
- `select()`, which picks variables based on their names [[2]](http://dplyr.tidyverse.org/reference/select.html)
- `filter()`, which picks variables based on their values [[3]](http://dplyr.tidyverse.org/reference/filter.html)
- `arrange()`, which reorders rows [[4]](http://dplyr.tidyverse.org/reference/arrange.html)

One can also use functions
`add_column` and `add_row` to add other data into a `tibble`.[[5]](https://tibble.tidyverse.org/reference/add_column.html) [[6]](https://tibble.tidyverse.org/reference/add_row.html)

Previous example with `tibble` would be:

In [94]:
cars_tibble %>% 
    filter(rowname == 'Dodge Challenger')

cars_tibble %>% 
    filter(rowname == 'Dodge Challenger') %>% 
    select('mpg')

cars_tibble %>% 
    select('cyl','disp')

cars_tibble %>% 
    filter(cyl == 6)

# Calculate l100km into a variable

cars_tibble <- cars_tibble %>%
    mutate(l100km = 235.214583/mpg)

# Obtain l100km from outside data

cars_tibble <- cars_tibble %>%
    add_column(l100km_2 = cars[['l100km']])

print(cars_tibble)

rowname,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Dodge Challenger,15.5,8,318,150,2.76,3.52,16.87,0,0,3,2


mpg
15.5


cyl,disp
6,160.0
6,160.0
4,108.0
6,258.0
8,360.0
6,225.0
8,360.0
4,146.7
4,140.8
6,167.6


rowname,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
Valiant,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
Merc 280,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4
Merc 280C,17.8,6,167.6,123,3.92,3.440,18.90,1,0,4,4
Ferrari Dino,19.7,6,145.0,175,3.62,2.770,15.50,0,1,5,6


# A tibble: 32 x 14
   rowname      mpg   cyl  disp    hp  drat    wt  qsec    vs    am  gear  carb
   <chr>      <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl> <dbl>
 1 Mazda RX4   21.0    6.  160.  110.  3.90  2.62  16.5    0.    1.    4.    4.
 2 Mazda RX4…  21.0    6.  160.  110.  3.90  2.88  17.0    0.    1.    4.    4.
 3 Datsun 710  22.8    4.  108.   93.  3.85  2.32  18.6    1.    1.    4.    1.
 4 Hornet 4 …  21.4    6.  258.  110.  3.08  3.22  19.4    1.    0.    3.    1.
 5 Hornet Sp…  18.7    8.  360.  175.  3.15  3.44  17.0    0.    0.    3.    2.
 6 Valiant     18.1    6.  225.  105.  2.76  3.46  20.2    1.    0.    3.    1.
 7 Duster 360  14.3    8.  360.  245.  3.21  3.57  15.8    0.    0.    3.    4.
 8 Merc 240D   24.4    4.  147.   62.  3.69  3.19  20.0    1.    0.    4.    2.
 9 Merc 230    22.8    4.  141.   95.  3.92  3.15  22.9    1.    0.    4.    2.
10 Merc 280    19.2    6.  168.  123.  3.92  3.44  18.3    1.    0.    4.    4.
# ... with 22 more r

It is important to note that all of the operations returned a `tibble`. This makes handling data much easier as you don't need to worry about converting objects.

## Functions

R is a highly procedural language. This means that most of the time you'll modify data through function calls. A function is defined into a variable with the assignment operator `<-` and the `function`-statement:

In [95]:
example_function <- function (argument, optional_argument='optional not set') {
    print(argument)
    print(optional_argument)
    return(0)
}

# Do not give an optional argument
example_function('test')
# Give arguments with positional placement
example_function('test2','optional set')
# Give arguments with keywords
example_function(optional_argument='optional_set3',argument='test3')
# Missing an argument returns an error, catch it with try()
print(try(example_function()))

[1] "test"
[1] "optional not set"


[1] 0

[1] "test2"
[1] "optional set"


[1] 0

[1] "test3"
[1] "optional_set3"


[1] 0

[1] "Error in print(argument) : \n  argument \"argument\" is missing, with no default\n"
attr(,"class")
[1] "try-error"
attr(,"condition")
<simpleError in print(argument): argument "argument" is missing, with no default>


Arguments are specified in the function definition. Ones with default values are considered optional. Positional arguments are evaluated from left to right. During function call one can also specify arguments by giving their names.

Output of the function call is returned with a call to the `return`-function. One can also use `invisible` to return the output only if it asked during function call. Returning output is especially important if you want to write a function that works with the Tidyverse pipe.

It is important to know that in R all objects are immutable. This means that if you provide a function with an object, R will copy this object once it is has been accessed (lazy loading) and any changes you make to this object are not done to the original object unless you assign the result to the original object.

In [107]:
x <- (1:10)

does_not_multiply_a_vector <- function (multiplier,inputvector) {
    inputvector <- multiplier * inputvector
    print('Input in function scope:')
    print(inputvector)
}

print('Input:')
print(x)
does_not_multiply_a_vector(2,x)
print('Input after function call:')
print(x)

does_multiply_a_vector <- function (multiplier,inputvector) {
    inputvector <- multiplier * inputvector
    print('Input in function scope:')
    print(inputvector)
    return(inputvector)
}

print('Input:')
print(x)
x <- does_multiply_a_vector(2,x)
print('Input after function call:')
print(x)

[1] "Input:"
 [1]  1  2  3  4  5  6  7  8  9 10
[1] "Input in function scope:"
 [1]  2  4  6  8 10 12 14 16 18 20
[1] "Input after function call:"
 [1]  1  2  3  4  5  6  7  8  9 10
[1] "Input:"
 [1]  1  2  3  4  5  6  7  8  9 10
[1] "Input in function scope:"
 [1]  2  4  6  8 10 12 14 16 18 20
[1] "Input after function call:"
 [1]  2  4  6  8 10 12 14 16 18 20


With Tinyverse verbs the previous example is of course more terse:

In [120]:
x_tibble <- tibble(x=(1:10))
print(x_tibble)
x_tibble <- x_tibble %>%
    mutate(x = x*2)

print(x_tibble)

# A tibble: 10 x 1
       x
   <int>
 1     1
 2     2
 3     3
 4     4
 5     5
 6     6
 7     7
 8     8
 9     9
10    10
# A tibble: 10 x 1
       x
   <dbl>
 1    2.
 2    4.
 3    6.
 4    8.
 5   10.
 6   12.
 7   14.
 8   16.
 9   18.
10   20.
